# Task 2

## Manual Ground Truth 
We pick the first question that contains a bug (Question.ID = 1) and all correct explanations to manually formulate a ground truth.

In [43]:
import pandas as pd

In [42]:
df = pd.read_csv('../../data/answerList_data.csv')

df['correctness'] = (((df['GroundTruth'] == 0.0) & (df['Answer.option'] == 'NO')) | (
            (df['GroundTruth'] == 1.0) & (df['Answer.option'] == 'YES'))).astype(int)

df_correct_question1 = df[(df['correctness'] == 1) & (df['Question.ID'] == 1)]
explanations = df_correct_question1['Answer.explanation']
for t in explanations:    
    print(t)


Minutes are set to -15; which is less then 0 and it throws illegal arg exception
In the code there is a check that 0 <= minutes < 60 and the minutesOffset is -15 which does not fall into these prarmeters thus throwing an Exception
There is a logical check for if minuteOffset is less than 0 or greater than 59 causing it to throw an exception because the value is out of bounds (negative number)
YES. The issue is on line 279 (as I explained in my first question; of which I misunderstood that I was only being asked about the specific issue; not generalized issue). On line 279 the variable "minutesOffSet" is parameterized to throw an exception if it is < 0 or > 59. Line 279 should read "if (minutesOffset < -59 || minutesOffset > 59) {" because now the method can take in the number of minutes as a negative and will allow the method to properly progress to invoke/call further methods such as those asked about in the two previous questions.
The variable "minutesOffset" is checked incorrectly b

Our manual ground truth is:

In [32]:
groundTruth = "The IF statement in line 279 checks whether minutesOffset is set to a value between 0 and 59. If not, an IllegalArgumentException is thrown. This is a bug because the minutesOffset may also be negative. The IF statement should check for the minutesOffset to be between -59 and 59."

---
Now we choose metrics to evaluate readability and semantic similarity between the manual ground truth and the explanations generated by the LLM.

## Readability

### Flesch Reading Ease and Automated Readability Index
We choose the Flesch Reading Ease metric and Automated Readability Index, as they are well-known readability metrics that are easy to compute and simple to interpret. The Flesch Reading Ease gives a score between 0 and 100, where a score between 60 and 80 is considered easy to read. The Automated Readability Index gives a score between 1 and 14, where for most readers an ideal score is between 7 and 9.


## Semantic Similarity

### BLEU
We selected BLEU because of its simplicity and our prior experience with it from the previous miniproject. BLEU is widely recognized as a standard metric for measuring semantic similarity and requires minimal computational effort.

### Cosine Similarity of Embeddings
A disadvantage of BLEU is that it does not consider the semantic meaning of the explanations. Therefore, we also compute the cosine similarity of the embeddings of the explanations and the ground truth. 



In [29]:
import nltk

def calculate_bleu(hyp, ref):
    hypothesis = hyp.split()
    reference = ref.split()
    return nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)

In [31]:

from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_cosine_similarity(hyp, ref):
    hyp_embedding = model.encode(hyp)
    ref_embedding = model.encode(ref)
    return np.dot(hyp_embedding, ref_embedding) / (np.linalg.norm(hyp_embedding) * np.linalg.norm(ref_embedding))



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
import textstat
def calculate_flesch_reading_ease(text):
    return textstat.flesch_reading_ease(text)

def calculate_automated_readability_index(text):
    return textstat.automated_readability_index(text)


# Run the experiments

In [40]:
# We use the prompt that performed better in our last mini project
def get_prompt_2(explanations):
        return f"""
Please summarize these reports of the same bug. 
Remove redundant information, but make sure that every information given in the explanations is retained that would be needed to fix the bug. 
Summarize the report of the bug concisely. Keep the summary as short as possible.

These are the reports:

{explanations}

"""

In [44]:
from openai import OpenAI
client = OpenAI(
    api_key=""
)

def query_llm(explanations):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": get_prompt_2(str(explanations)),
        }
    ],
    model="gpt-4o-mini",
    )

    return chat_completion.choices[0].message.content

In [ ]:
# sample 1 to n explanations, 3 times and take average
# plot metric scores
# find good threshold to decide how many explanations to merge

In [ ]:

import matplotlib.pyplot as plt
def sample_explanations(explanations, n):
    return explanations.sample(n)

# plot metric scores
def plot_metric_scores(metric_scores):
    plt.plot(metric_scores)
    plt.show()

    